## Import Modules

In [3]:
import pandas as pd
import numpy as np

import os
import json

import shutil
from pathlib import Path
import yaml
from concurrent.futures import ThreadPoolExecutor
from tqdm.notebook import tqdm

import albumentations as A
import cv2

import torch
from ultralytics import settings
from ultralytics import YOLO



# Module Config

In [4]:
# Ultralytics settings
print(settings) 

# GPU Utilisation
print(torch.cuda.is_available())
device = torch.device("cuda")

{'settings_version': '0.0.4', 'datasets_dir': 'D:\\Tom\\GitHub\\Third Year Project\\datasets', 'weights_dir': 'D:\\Tom\\GitHub\\Third Year Project\\Intruder-Aircraft-Detection\\weights', 'runs_dir': 'D:\\Tom\\GitHub\\Third Year Project\\Intruder-Aircraft-Detection\\runs', 'uuid': 'fdb5c10788ffaa41a9047dc764dd8a0a3287d6bec12c0c66234ac745c0366efa', 'sync': True, 'api_key': '', 'openai_api_key': '', 'clearml': True, 'comet': True, 'dvc': True, 'hub': True, 'mlflow': True, 'neptune': True, 'raytune': True, 'tensorboard': True, 'wandb': True}
True


## Load Dataset

In [5]:
# Base paths for the images and labels
train_images_path = 'datasets/AVOIDDS/images/train'
train_labels_path = 'datasets/AVOIDDS/labels/train'
val_images_path = 'datasets/AVOIDDS/images/valid'
val_labels_path = 'datasets/AVOIDDS/labels/valid'

# Load the metadata
metadata_path = 'datasets/AVOIDDS/metadata.json'
with open(metadata_path, 'r') as file:
    metadata = json.load(file)

# Function to create a DataFrame from images and labels
def create_dataframe(images_path, labels_path, metadata):

    # List all files in the directories
    image_files = [f for f in sorted(os.listdir(images_path)) if f.endswith('.jpg')]
    label_files = [f for f in sorted(os.listdir(labels_path)) if f.endswith('.txt')]
    
    # Create tempory DataFrame so that final dataframe is in correct order
    temp_df = pd.DataFrame({
        'image_path': [str(images_path + '/' + file) for file in image_files],
        'label_path': [str(labels_path + '/' + file) for file in label_files],
    })

    # Extract image indices to match with metadata
    df = pd.DataFrame()
    df['imageID'] = temp_df['image_path'].apply(lambda x: int(os.path.splitext(os.path.basename(x))[0]))

    # Add image and label paths to final dataframe
    df['image_path'] = temp_df['image_path']
    df['label_path'] = temp_df['label_path']
 
    # Add metadata to each image entry
    for key, value in metadata.items():
        if '.' in key:  # Key represents a range
            start, end = map(int, key.split('.'))
            df.loc[df['imageID'].between(start, end), 'metadata'] = json.dumps(value)

    # Convert the JSON strings in 'metadata' to dictionaries
    df['metadata'] = df['metadata'].apply(json.loads)

    # Expand the 'metadata' column into separate columns
    metadata_df = pd.json_normalize(df['metadata'])
    
    # Concatenate the expanded metadata back to the original DataFrame
    full_df = pd.concat([df.drop(['metadata'], axis=1), metadata_df], axis=1)

    return full_df

# Create the DataFrames for the train and validation sets
train_df = create_dataframe(train_images_path, train_labels_path, metadata)
valid_df = create_dataframe(val_images_path, val_labels_path, metadata)

train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64800 entries, 0 to 64799
Data columns (total 23 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   imageID      64800 non-null  int64  
 1   image_path   64800 non-null  object 
 2   label_path   64800 non-null  object 
 3   location     64800 non-null  object 
 4   enrange      64800 non-null  float64
 5   urange       64800 non-null  float64
 6   weather      64800 non-null  int64  
 7   daystart     64800 non-null  float64
 8   dayend       64800 non-null  float64
 9   num_train    64800 non-null  int64  
 10  num_valid    64800 non-null  int64  
 11  append       64800 non-null  bool   
 12  datasetname  64800 non-null  object 
 13  ac           64800 non-null  object 
 14  allweather   64800 non-null  bool   
 15  newac        64800 non-null  bool   
 16  own_h        64800 non-null  object 
 17  own_p_max    64800 non-null  float64
 18  own_r_max    64800 non-null  float64
 19  intr

# Create sub datasets

In [6]:

def create_dataset(dataset_name, filtered_train_df, filtered_valid_df, class_names=['aircraft'], dataset_dir="datasets/"):
    # Construct the base path using the dataset name
    dataset_dir = Path(dataset_dir) / dataset_name
    
    # Define subdirectories for images and labels
    images_dir = dataset_dir / 'images'
    labels_dir = dataset_dir / 'labels'
    
    # Create the necessary directories
    for subdir in ['train', 'valid']:
        (images_dir / subdir).mkdir(parents=True, exist_ok=True)
        (labels_dir / subdir).mkdir(parents=True, exist_ok=True)
    
    print("created directories")

    # Function to copy files from the DataFrame to the respective directories
    #def copy_files(df, img_dest_dir, label_dest_dir):
    #    def copy_file(row):
    #        shutil.copy(row['image_path'], img_dest_dir / f"{Path(row['image_path']).name}")
    #        shutil.copy(row['label_path'], label_dest_dir / f"{Path(row['label_path']).name}")
    #        
    #    # Wrap iterrows with tqdm for a progress bar
    #    list_of_rows = list(df.iterrows())
    #    for _ in tqdm(range(len(list_of_rows)), desc='Copying files'):
    #        # Use ThreadPoolExecutor to copy files in parallel
    #        with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
    #            executor.map(copy_file, [row for _, row in list_of_rows])
    
    def copy_files(df, img_dest_dir, label_dest_dir):
        # Initialize tqdm progress bar with the total length of the DataFrame
        pbar = tqdm(total=len(df), desc='Copying files')
        
        for _, row in df.iterrows():
            # Copy files
            shutil.copy(row['image_path'], img_dest_dir / f"{Path(row['image_path']).name}")
            shutil.copy(row['label_path'], label_dest_dir / f"{Path(row['label_path']).name}")
            # Update the progress bar
            pbar.update(1)
        # Close the progress bar after completion
        pbar.close()

    # Copy the files for the train and validation sets
    copy_files(filtered_train_df, images_dir / 'train', labels_dir / 'train')
    print("copied train files")
    copy_files(filtered_valid_df, images_dir / 'valid', labels_dir / 'valid')
    print("copied valid files")
    
    # Create the .yaml configuration file for the dataset
    yaml_content = {
        'path': str(dataset_dir),  # Optionally include the path in the YAML
        'train': str(images_dir / 'train'),
        'val': str(images_dir / 'valid'),
        #'nc': len(class_names),
        'names': class_names
    }
    
    # Create yaml file
    yaml_path = dataset_dir / f"{dataset_name}.yaml"
    with open(yaml_path, 'w') as file:
        yaml.dump(yaml_content, file, sort_keys=False)
    
    print(f"Dataset '{dataset_name}' created at {dataset_dir}")


In [ ]:
cessna_train_df = train_df[train_df['ac'] == 'Cessna Skyhawk']
cessna_valid_df = valid_df[valid_df['ac'] == 'Cessna Skyhawk']

dataset_name = 'cessna_dataset'

create_dataset(dataset_name, cessna_train_df, cessna_valid_df)


# Augmentation Methods

In [ ]:
def read_label(label_path):

    # Placeholder for reading the label file in YOLO format
    pass

def write_label(label_path, bboxes, image_shape):
    
    # Placeholder for writing the label file in YOLO format
    pass

def augment_image(image_path, label_path, augmentation):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    bboxes = read_label(label_path)

    # Note that bboxes should be normalized (i.e., in the range [0, 1])
    transformed = augmentation(image=image, bboxes=bboxes)
    transformed_image = transformed['image']
    transformed_bboxes = transformed['bboxes']

    # Denormalize bboxes here if your write_label function expects that
    write_label(label_path, transformed_bboxes, transformed_image.shape)

    cv2.imwrite(image_path, cv2.cvtColor(transformed_image, cv2.COLOR_RGB2BGR))

# Zoom in/out
def zoom(image_path, label_path, min_zoom=0.8, max_zoom=1.2):
    augmentation = A.Compose([
        A.RandomScale(scale_limit=(min_zoom - 1, max_zoom - 1), p=1.0)
    ], bbox_params=A.BboxParams(format='yolo', label_fields=[]))
    
    augment_image(image_path, label_path, augmentation)


# Horizontal and Vertical Reflection
def reflection(image_path, label_path):
    augmentation = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5)
    ], bbox_params=A.BboxParams(format='yolo', label_fields=[]))
    
    augment_image(image_path, label_path, augmentation)

# Rotation
def rotate(image_path, label_path, angle_limit=90):
    augmentation = A.Compose([
        A.Rotate(limit=angle_limit, p=1.0)
    ], bbox_params=A.BboxParams(format='yolo', label_fields=[]))
    
    augment_image(image_path, label_path, augmentation)

# Contrast and Brightness
def adjust_contrast_brightness(image_path, label_path, brightness_limit=0.2, contrast_limit=0.2):
    augmentation = A.Compose([
        A.RandomBrightnessContrast(brightness_limit=brightness_limit, contrast_limit=contrast_limit, p=1.0)
    ])
    
    augment_image(image_path, label_path, augmentation)

# Histogram Equalization
def histogram_equalization(image_path, label_path):
    augmentation = A.Compose([
        A.HistogramMatching(p=1.0)
    ])
    
    augment_image(image_path, label_path, augmentation)

# White Balance (using FancyPCA for color augmentation which can simulate white balance shifts)
def white_balance(image_path, label_path):
    augmentation = A.Compose([
        A.FancyPCA(alpha=0.1, p=1.0)
    ])
    
    augment_image(image_path, label_path, augmentation)

# Sharpen
def sharpen(image_path, label_path):
    augmentation = A.Compose([
        A.Sharpen(alpha=(0.2, 0.5), lightness=(0.5, 1.0), p=1.0)
    ])
    
    augment_image(image_path, label_path, augmentation)

# Gaussian Noise
def gaussian_noise(image_path, label_path, var_limit=(10.0, 50.0)):
    augmentation = A.Compose([
        A.GaussNoise(var_limit=var_limit, p=1.0)
    ])
    
    augment_image(image_path, label_path, augmentation)



# Test Parameters

In [ ]:
# parameters for each specfic test














# Training

# Own Implementation of Object Detection System